# Pytorch Neural Networks Models

In [1]:
import torch 
import torchvision
from torch import nn, optim
from torch.autograd import Variable
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
from torch.autograd import Variable
import numpy as np 
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix 

Using TensorFlow backend.


In [2]:
tr = transforms.Compose([transforms.ToTensor()])

train_digits = torchvision.datasets.MNIST(root='./', train=True, download=False, transform=tr )
test_digits = torchvision.datasets.MNIST(root='./', train=False, download=False, transform=tr )

train, val = random_split(train_digits, [55000, 5000])

train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_digits, batch_size=64, shuffle=True)

# Pytorch Neural Network

In [3]:
class Nnet(nn.Module):
    def __init__(self):
        super(Nnet, self).__init__()
        self.lin1 = nn.Linear(28*28, 128)
        self.lin2 = nn.Linear(128, 64)
        self.lin3 = nn.Linear(64, 64)
        self.lin4 = nn.Linear(64, 32)
        self.softmax = nn.Softmax()
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, images):
        x = images.view(-1, 28*28)
        x = self.relu(self.lin1(x))
        x = self.sigmoid(self.lin2(x))
        x = self.relu(self.lin3(x))
        x = self.softmax(self.lin4(x))

        return x

In [4]:
model = Nnet()
loss_func = nn.CrossEntropyLoss()
params = model.parameters()
opt = optim.Adam(params, lr=0.001)
epochs = 1250
iters = 0

In [5]:
for epoch in range(epochs):
    losses = list()
    accuracies = list()
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)

        output = model(images)

        model.zero_grad()
        loss = loss_func(output, labels)
        loss.backward() 

        opt.step() 

        losses.append(loss.item())
        accuracies.append(labels.eq(output.detach().argmax(dim=1)).float().mean())

        print(f'Epoch {epoch + 1 }', end= ', ' )
        print(f'training loss: {torch.tensor(losses).mean():.2f}', end=',')
        print(f'training accuracy: {torch.tensor(accuracies).mean():.2f}')

        iters += 1

        break

.75
Epoch 892, training loss: 2.80,training accuracy: 0.72
Epoch 893, training loss: 2.77,training accuracy: 0.77
Epoch 894, training loss: 2.76,training accuracy: 0.80
Epoch 895, training loss: 2.71,training accuracy: 0.83
Epoch 896, training loss: 2.74,training accuracy: 0.80
Epoch 897, training loss: 2.82,training accuracy: 0.70
Epoch 898, training loss: 2.70,training accuracy: 0.83
Epoch 899, training loss: 2.68,training accuracy: 0.84
Epoch 900, training loss: 2.75,training accuracy: 0.77
Epoch 901, training loss: 2.73,training accuracy: 0.83
Epoch 902, training loss: 2.70,training accuracy: 0.81
Epoch 903, training loss: 2.71,training accuracy: 0.83
Epoch 904, training loss: 2.73,training accuracy: 0.80
Epoch 905, training loss: 2.73,training accuracy: 0.80
Epoch 906, training loss: 2.71,training accuracy: 0.83
Epoch 907, training loss: 2.77,training accuracy: 0.77
Epoch 908, training loss: 2.70,training accuracy: 0.83
Epoch 909, training loss: 2.73,training accuracy: 0.81
Epoch 

In [6]:
@torch.no_grad()
def get_eval(model, test_loader):
    predictions = torch.tensor([])
    testLabels = torch.tensor([])
    for batch in test_loader:
        images, label = batch

        preds = model(images)
        predictions = torch.cat(
            (predictions, preds)
            ,dim=0
        )
        testLabels = torch.cat(
            (testLabels, label)
            ,dim=0
        )
    predictions = np.array(predictions)
    testLabels = np.array(testLabels)
    return predictions, testLabels

with torch.no_grad():
    predictions, testLabels = get_eval(model, test_loader)

In [7]:
testLabels =  to_categorical(testLabels)
testLabels = np.argmax(testLabels, axis=1)
predictions = np.argmax(predictions,axis=1)

In [8]:
print("Prediction: ", predictions[:20])
print("Actual: ",testLabels[:20])

Prediction:  [3 3 1 9 9 8 1 8 0 2 7 2 0 2 0 6 3 6 6 0]
Actual:  [3 3 1 9 4 4 1 3 0 8 7 2 0 2 0 6 3 6 6 0]


In [9]:
print(classification_report(testLabels, predictions))

              precision    recall  f1-score   support

           0       0.93      0.98      0.95       980
           1       0.96      0.99      0.97      1135
           2       0.90      0.90      0.90      1032
           3       0.90      0.91      0.90      1010
           4       0.00      0.00      0.00       982
           5       0.87      0.84      0.86       892
           6       0.92      0.93      0.93       958
           7       0.91      0.94      0.92      1028
           8       0.86      0.89      0.88       974
           9       0.50      0.92      0.65      1009

    accuracy                           0.83     10000
   macro avg       0.78      0.83      0.80     10000
weighted avg       0.78      0.83      0.80     10000



In [10]:
print(confusion_matrix(testLabels, predictions))

[[ 960    0    1    1    0    6    6    3    2    1]
 [   0 1119    2    4    0    1    3    1    5    0]
 [  14    9  931   15    0    5   11   17   24    6]
 [   1    1   24  919    0   20    0   18   25    2]
 [   2    4   10    0    0    3   42   28   25  868]
 [  13    2   12   55    0  749    7    5   41    8]
 [  26    3   11    0    0   17  895    0    4    2]
 [   2    9   24    3    0    1    0  968    0   21]
 [   4   14   14   17    0   36    8    6  869    6]
 [   9    5    0   10    0   19    4   19   13  930]]


# Pytorch Dropout Neural Network

In [11]:
class DNet(nn.Module):
    def __init__(self):
        super(DNet, self).__init__()
        self.l1 = nn.Linear(28 * 28, 64)
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64, 32)
        self.do = nn.Dropout(0.1)
        self.softmax = nn.Softmax()
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = x.view(-1, 28*28)
        l1 = self.relu(self.l1(x))
        l2 = self.sigmoid(self.l2(l1))
        do = self.do(l2 + l1)
        l3 = self.softmax(self.l3(do))
        return l3
    

d_model = DNet()

In [22]:
d_model = DNet()
d_loss_func = nn.CrossEntropyLoss()
d_params = d_model.parameters()
d_opt = optim.Adam(d_params, lr=0.001)
d_epochs = 750
d_iters = 0

In [23]:
for d_epoch in range(d_epochs):
    d_losses = list()
    d_accuracies = list()
    d_model.train()
    for i, (d_images, d_labels) in enumerate(train_loader):
        d_images = Variable(d_images)
        d_labels = Variable(d_labels)

        d_output = d_model(d_images)

        d_model.zero_grad()
        d_loss = d_loss_func(d_output, d_labels)
        d_loss.backward() 

        d_opt.step() 

        d_losses.append(d_loss.item())
        d_accuracies.append(d_labels.eq(d_output.detach().argmax(dim=1)).float().mean())

        print(f'Epoch {d_epoch + 1 }', end= ', ' )
        print(f'training loss: {torch.tensor(d_losses).mean():.2f}', end=',')
        print(f'training accuracy: {torch.tensor(d_accuracies).mean():.2f}')

        d_iters += 1
        break

loss: 2.60,training accuracy: 0.94
Epoch 388, training loss: 2.79,training accuracy: 0.75
Epoch 389, training loss: 2.65,training accuracy: 0.91
Epoch 390, training loss: 2.78,training accuracy: 0.75
Epoch 391, training loss: 2.67,training accuracy: 0.88
Epoch 392, training loss: 2.67,training accuracy: 0.89
Epoch 393, training loss: 2.67,training accuracy: 0.86
Epoch 394, training loss: 2.76,training accuracy: 0.77
Epoch 395, training loss: 2.69,training accuracy: 0.88
Epoch 396, training loss: 2.71,training accuracy: 0.81
Epoch 397, training loss: 2.68,training accuracy: 0.86
Epoch 398, training loss: 2.75,training accuracy: 0.78
Epoch 399, training loss: 2.71,training accuracy: 0.81
Epoch 400, training loss: 2.82,training accuracy: 0.70
Epoch 401, training loss: 2.70,training accuracy: 0.86
Epoch 402, training loss: 2.71,training accuracy: 0.84
Epoch 403, training loss: 2.74,training accuracy: 0.81
Epoch 404, training loss: 2.74,training accuracy: 0.80
Epoch 405, training loss: 2.71

# Convolutional Neural Network Predictions

In [24]:
with torch.no_grad():
    d_predictions, d_testLabels = get_eval(d_model, test_loader)
    
d_testLabels =  to_categorical(d_testLabels)
d_testLabels = np.argmax(d_testLabels, axis=1)
d_predictions = np.argmax(d_predictions,axis=1)

In [25]:
print("Prediction: ", d_predictions[:20])
print("Actual: ",d_testLabels[:20])

Prediction:  [0 2 1 8 5 7 0 9 3 7 9 6 5 1 4 2 7 2 4 1]
Actual:  [0 2 1 8 5 7 0 9 3 7 9 6 5 1 4 2 7 2 4 1]


# Classification Report and Confusion Matrix

In [26]:
print(classification_report(d_testLabels, d_predictions))

              precision    recall  f1-score   support

           0       0.93      0.98      0.95       980
           1       0.96      0.97      0.97      1135
           2       0.94      0.87      0.90      1032
           3       0.93      0.87      0.90      1010
           4       0.88      0.93      0.91       982
           5       0.90      0.83      0.86       892
           6       0.94      0.93      0.94       958
           7       0.93      0.90      0.92      1028
           8       0.75      0.84      0.79       974
           9       0.88      0.89      0.89      1009

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.91      0.90      0.90     10000



In [27]:
print(confusion_matrix(d_testLabels, d_predictions))

[[ 958    0    0    1    1    5    7    2    6    0]
 [   0 1104    5    1    1    3    5    1   15    0]
 [  16    2  895   12   17    0    8   18   61    3]
 [   3    0   17  877    1   34    2   12   55    9]
 [   1    3    3    0  914    1    6    2   14   38]
 [  11    3    4   25   15  740   12    9   67    6]
 [  15    3    0    0   16   16  892    0   16    0]
 [   3   11   15    3    9    0    0  930   19   38]
 [  12   18    8   18   22   23   14   12  821   26]
 [  11    9    1    2   39    3    1   13   27  903]]
